# 0. Configuration

In [62]:
# links to shared data MovieLens
# source on kaggle: https://www.kaggle.com/code/quangnhatbui/movie-recommender/data
MOVIES_METADATA_URL = 'https://drive.google.com/file/d/19g6-apYbZb5D-wRj4L7aYKhxS-fDM4Fb/view?usp=share_link'
RATINGS_SMALL_URL = 'https://drive.google.com/file/d/1BlZfCLLs5A13tbNSJZ1GPkHLWQOnPlE4/view?usp=share_link'

# 1. Modules and functions

In [63]:
import re
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from ast import literal_eval
from pymystem3 import Mystem
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import warnings
warnings.filterwarnings('ignore')

# download stop words beforehand
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikitasenyatkin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1.1. Helper functions to avoid copypaste

In [64]:
def read_csv_from_gdrive(url):
    """
    gets csv data from a given url (taken from file -> share -> copy link)
    :url: example https://drive.google.com/file/d/1BlZfCLLs5A13tbNSJZ1GPkHLWQOnPlE4/view?usp=share_link
    """
    file_id = url.split('/')[-2]
    file_path = 'https://drive.google.com/uc?export=download&id=' + file_id
    data = pd.read_csv(file_path)

    return data

In [65]:
# init lemmatizer to avoid slow performance
mystem = Mystem() 

def word_tokenize_clean(doc: str, stop_words: list):
    '''
    tokenize from string to list of words
    '''

    # split into lower case word tokens \w lemmatization
    tokens = list(set(mystem.lemmatize(doc.lower())))
  
    # remove tokens that are not alphabetic (including punctuation) and not a stop word
    tokens = [word for word in tokens if word.isalpha() and not word in stop_words \
              not in list(punctuation)]
    return tokens

# 2. Main

## 2.1. Data Preparation

In [66]:
# read csv information about films etc
movies_metadata = read_csv_from_gdrive(MOVIES_METADATA_URL)
movies_metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [67]:
# let's see what columns we have
movies_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

To get accurate results we need to preprocess text a bit. The pipeline will be as follows:

- Filter only necessary columns from movies_metadada : id, original_title, overview;
- Define `model_index` for model to match back with `id` column;
- Text cleaning: removing stopwords & punctuation, lemmatization for further tokenization and tagged document creatin required for gensim.Doc2Vec

In [68]:
# filter cols
sample = movies_metadata[['id', 'original_title', 'overview']].copy()
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              45466 non-null  object
 1   original_title  45466 non-null  object
 2   overview        44512 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [69]:
# as you see from above, we have missing overview in some cases -- let's fill it with the original title
sample.loc[sample['overview'].isnull(), 'overview'] = sample.loc[sample['overview'].isnull(), 'original_title']
sample.isnull().sum()

id                0
original_title    0
overview          0
dtype: int64

In [70]:
# define model_index and make it as string
sample = sample.reset_index().rename(columns = {'index': 'model_index'})
sample['model_index'] = sample['model_index'].astype(str)

In [71]:
# create mapper with title and model_idnex to use it further in evaluation
movies_inv_mapper = dict(zip(sample['original_title'].str.lower(), sample['model_index'].astype(int)))

In [72]:
# preprocess by removing non-character data, stopwords
tags_corpus = sample['overview'].values
tags_corpus = [re.sub('-[!/()0-9]', '', x) for x in tags_corpus]
stop_words = stopwords.words('english')

tags_doc = [word_tokenize_clean(description, stop_words) for description in tags_corpus]
tags_doc[:20]

[['put',
  'scene',
  'andy',
  'separate',
  'onto',
  'circumstances',
  'brings',
  'lightyear',
  'place',
  'heart',
  'buzz',
  'toys',
  'owner',
  'afraid',
  'aside',
  'learns',
  'happily',
  'birthday',
  'room',
  'woody',
  'differences',
  'plots',
  'live',
  'losing',
  'eventually',
  'led',
  'duo'],
 ['world',
  'magical',
  'invite',
  'hope',
  'rhinoceroses',
  'monkeys',
  'enchanted',
  'creatures',
  'finish',
  'living',
  'three',
  'unwittingly',
  'alan',
  'game',
  'door',
  'years',
  'giant',
  'freedom',
  'running',
  'adult',
  'peter',
  'trapped',
  'room',
  'siblings',
  'terrifying',
  'proves',
  'discover',
  'judy',
  'risky',
  'board',
  'opens',
  'evil',
  'find',
  'inside'],
 ['family',
  'fish',
  'interested',
  'alarming',
  'restaurant',
  'scare',
  'away',
  'reignites',
  'time',
  'locals',
  'door',
  'meanwhile',
  'feud',
  'shop',
  'hot',
  'next',
  'local',
  'max',
  'ancient',
  'john',
  'bait',
  'sultry',
  'neighbo

In [73]:
# prepare data as model input for Word2Vec
## it takes some time to execute
tags_doc = [TaggedDocument(words = word_tokenize_clean(D, stop_words), tags = [str(i)]) for i, D in enumerate(tags_corpus)]

In [74]:
# let's check what do we have
## tag = movie index
tags_doc[1]

TaggedDocument(words=['world', 'magical', 'invite', 'hope', 'rhinoceroses', 'monkeys', 'enchanted', 'creatures', 'finish', 'living', 'three', 'unwittingly', 'alan', 'game', 'door', 'years', 'giant', 'freedom', 'running', 'adult', 'peter', 'trapped', 'room', 'siblings', 'terrifying', 'proves', 'discover', 'judy', 'risky', 'board', 'opens', 'evil', 'find', 'inside'], tags=['1'])

# 2.2. Model Training and Evaluation

In [75]:
VEC_SIZE = 50
ALPHA = .02
MIN_ALPHA = .00025
MIN_COUNT = 5
EPOCHS = 20

In [76]:
# initialize
model = Doc2Vec(vector_size = VEC_SIZE,
                alpha = ALPHA, 
                min_alpha = MIN_ALPHA,
                min_count = MIN_COUNT,
                dm = 0)

In [77]:
# generate vocab from all tag docs
model.build_vocab(tags_doc)

In [78]:
# train model
model.train(tags_doc,
            total_examples = model.corpus_count,
            epochs = EPOCHS)

## 2.3. Evaluate the model

Let's assume that we watched movie `batman` and based on that generate recommendation similar to it's description.

To do that we need
- To extract movie id from `movies_inv_mapper` we created to map back titles from model output
- Load embeddings from trained model
- Use built-in most_similar() method to get most relevant recommendations based on film embedding
- Finally, map title names for sense-check

In [79]:
# get id
movie_id = movies_inv_mapper['batman']
movie_id

8603

In [80]:
# load trained embeddings 
movies_vectors = model.dv.vectors

In [81]:
movie_embeddings = movies_vectors[movie_id]

In [82]:
movie_embeddings

array([-8.0160208e-02, -1.8253161e-01,  2.7860367e-01, -1.9340388e-02,
       -2.6517289e-02,  1.8640238e-01, -3.3289498e-01, -6.1052497e-02,
       -3.5598734e-01, -3.8000790e-04, -1.7075939e-01, -1.1591378e-01,
        7.9556266e-05, -2.0348647e-01,  2.0137224e-01, -9.6725069e-02,
        2.0268382e-01, -1.6914688e-01,  1.5876643e-01, -2.0017324e-01,
        9.4643004e-02,  1.4631657e-01, -3.1319104e-02,  1.7299244e-01,
        1.5535557e-01,  2.1169420e-01, -1.1648132e-01, -1.2777744e-01,
        3.0593607e-01, -2.5232095e-01,  2.5669878e-02,  6.3207977e-02,
       -4.3362007e-02,  7.3174194e-02, -3.5317650e-01,  2.9851297e-01,
       -1.5897123e-02, -7.3791169e-02,  1.8562043e-01,  2.0299160e-01,
        1.6092224e-01, -2.9811222e-02, -3.5844274e-02,  1.4017850e-01,
        1.3455860e-02, -3.6246192e-01,  6.1877277e-02,  2.6866566e-02,
       -1.2715700e-01,  4.0484986e-01], dtype=float32)

In [83]:
# get recommendations
similars = model.docvecs.most_similar(positive = [movie_embeddings], topn = 20)
output = pd.DataFrame(similars, columns = ['model_index', 'model_score'])
output.head()

,model_index,model_score
0,8603,1.000000
1,5713,0.959517
2,19227,0.958552
3,43461,0.953251
4,13835,0.952664


In [84]:
# reverse values and indices to map names in dataframe
name_mapper = {v: k for k, v in movies_inv_mapper.items()}

In [85]:
output['title_name'] = output['model_index'].astype(int).map(name_mapper)
output


,model_index,model_score,title_name
0,8603,1.000000,batman
1,5713,0.959517,rollover
2,19227,0.958552,carbon nation
3,43461,0.953251,megafault
4,13835,0.952664,k2
5,6816,0.952464,death machine
6,43165,0.950795,the zookeeper's wife
7,7772,0.950565,this island earth
8,10227,0.949204,stealth
9,1045,0.948662,sleeper


# TODO

- Add `original_title`, `keywords`, `tagline` and other metadata to train sample and then retrain embeddings;
- Make visualization of embeddings with links of films with each other;
- Compare results with the embeddings we created in lecture
- Write function get_recommendations() which takes arguments we used 2.3., but such that we can use embeddings of several watched films to get recommendations

# Appendix

Here, we wrap up all pipeline into functions to re-use if needed and it is just prettier to code this way :)

## Making personal rekkos (building get_recommendations() function that will give personal rekkos for each user with filter on watched films)

In [86]:
def get_clean_tags_array(agg_tags: pd.DataFrame,
                         text_col = 'overview'):
    '''text preprocessing
    '''
    tags_corpus = agg_tags[text_col].values
    tags_corpus = [re.sub('-[!/()0-9]', '', x) for x in tags_corpus]
    stop_words = stopwords.words('english')


    # preprocess corpus of movie tags before feeding it into Doc2Vec model
    tags_doc = [TaggedDocument(words = word_tokenize_clean(D, stop_words), tags = [str(i)]) for i, D in enumerate(tags_corpus)]

    return tags_doc


In [87]:
def train_embeddings(tags_doc: np.array,
                     epochs = 20,
                     vec_size = 50,
                     alpha = .02,
                     min_alpha =  0.00025,
                     min_count = 5,
                     save_path: str = None):
    """
    fit doc2vec model to prepared corpus
    :tags_doc: result of get_clean_tags_array()
    :max_epocs: int
    :vec_size: int
    :alpha: float
    """
    #initialize
    model = Doc2Vec(vector_size = vec_size,
                    alpha = alpha, 
                    min_alpha = min_alpha,
                    min_count = min_count,
                    dm = 0)
    
    #generate vocab from all tag docs
    model.build_vocab(tags_doc)
    
    #train model
    model.train(tags_doc,
                total_examples = model.corpus_count,
                epochs = epochs)
    
    #save model to dir
    if save_path:
        model.save(f'{save_path}/d2v_model.pkl')
    
    return model

In [88]:
tags_doc = get_clean_tags_array(sample)


In [89]:
model = train_embeddings(tags_doc)

lets load interactions data

In [90]:
interactions = read_csv_from_gdrive(RATINGS_SMALL_URL)
interactions['movieId'] = interactions['movieId'].astype(str)
movies_metadata.rename(columns = {'id': 'movieId'}, inplace = True)
interactions_filtered = interactions.loc[interactions['movieId'].isin(sample['model_index'])]
print(interactions.shape, interactions_filtered.shape)

(100004, 4) (88299, 4)


In [91]:
# create users input
users = interactions_filtered[['userId']].drop_duplicates().reset_index(drop = True)

In [92]:
known_items = interactions_filtered.groupby('userId')['movieId'].apply(list).to_dict()

users['watched_movies'] = users['userId'].map(known_items)
users.head(5)

,userId,watched_movies
0,1,"[31, 1029, 1061, 1129, 1172, 1263, 1287, 1293,..."
1,2,"[10, 17, 39, 47, 50, 52, 62, 110, 144, 150, 15..."
2,3,"[60, 110, 247, 267, 296, 318, 355, 356, 377, 5..."
3,4,"[10, 34, 112, 141, 153, 173, 185, 260, 289, 29..."
4,5,"[3, 39, 104, 141, 150, 231, 277, 344, 356, 364..."


In [93]:
movie_vectors = model.dv.vectors

In [94]:
# Defining a function to get the embeddings of a list of movies
def get_movie_embeddings(movie_list):
    embeddings = [movie_vectors[int(movie_id)] for movie_id in movie_list]
    # Remove None values
    embeddings = [x for x in embeddings if x is not None]
    return embeddings


users['watched_movies_embeddings'] = users['watched_movies'].apply(get_movie_embeddings)


In [95]:
users.head(5)

,userId,watched_movies,watched_movies_embeddings
0,1,"[31, 1029, 1061, 1129, 1172, 1263, 1287, 1293,...","[[0.03041095, -0.096139565, 0.124947004, 0.037..."
1,2,"[10, 17, 39, 47, 50, 52, 62, 110, 144, 150, 15...","[[-0.15852125, -0.28294635, 0.26542282, -0.050..."
2,3,"[60, 110, 247, 267, 296, 318, 355, 356, 377, 5...","[[-0.07212124, -0.043663364, 0.28096688, 0.014..."
3,4,"[10, 34, 112, 141, 153, 173, 185, 260, 289, 29...","[[-0.15852125, -0.28294635, 0.26542282, -0.050..."
4,5,"[3, 39, 104, 141, 150, 231, 277, 344, 356, 364...","[[-0.05128915, -0.05324564, 0.34414858, -0.062..."


In [96]:
## lets calculate mean embeddings for each user:

mean_embeddings = []
for embedding_list in users['watched_movies_embeddings']:
    embeddings = np.array(embedding_list)
    # Take the mean of the embeddings for movies that the user has watched
    user_embedding = np.mean(embeddings, axis=0)
    mean_embeddings.append(user_embedding)
    
# Add the mean_embeddings column to dataframe
users['mean_embeddings'] = mean_embeddings

In [97]:
#mapping titles and ids for furher function
name_mapper = dict(zip(sample['model_index'], sample['original_title']))

## Function that creates recommendations for each user

In [98]:
def rekkos(data: pd.DataFrame, number_of_samples: int):
    similars = model.docvecs.most_similar(positive=data, topn = number_of_samples)
    return [name_mapper.get(movie_id[0], movie_id[0]) for movie_id in similars]

let's calculate maximum films that we should predict to recommend every user fresh films:

In [99]:
lst_max = []
for row in users.index:
    lst_max.append(len(users['watched_movies'][row]))

max_films = max(lst_max)+20
max_films


1888

In [100]:
def watched_films_filter(df: pd.DataFrame, number_of_samples: int):
    """
    calculates mean rating to define popular titles with taking to the account watched films
    """
    popular_titles = df['rekkos']
    personal_rekkos = {}
    for key in known_items.keys():
        popular_titles = df['rekkos'][key-1]
        list_of_unknown_films = [x for x in popular_titles if x not in known_items[key]]
        personal_rekkos[key] = list(list_of_unknown_films[:number_of_samples])
    return personal_rekkos


## FINAL FUNCTION 

In [101]:
def get_recommendatios(data: pd.DataFrame, 
                       number_of_samples: int,
                       item_column: str):
    
    '''function that returns dataframe with recommended films for each user based on their previous history of watches.
    :data : pd.DataFrame 
    :number_of_samples : number of films needed to be predicted,
    :item_column : column with watched films in df.
    '''

    # creating embeddings for each movie
    data['watched_movies_embeddings'] = data[item_column].apply(get_movie_embeddings)

    #counting mean embeddings for watched_films
    mean_embeddings = []
    for embedding_list in data['watched_movies_embeddings']:
        embeddings = np.array(embedding_list)
        # Take the mean of the embeddings for movies that the user has watched
        user_embedding = np.mean(embeddings, axis=0)
        mean_embeddings.append(user_embedding)
    
    # Adding the mean_embeddings column to dataframe
    data['mean_embeddings'] = mean_embeddings

    #receiving rekkos
    data['rekkos'] = data['mean_embeddings'].apply(rekkos, number_of_samples=max_films)

    #filtering watched films 
    personal_rekkos = watched_films_filter(data, number_of_samples=number_of_samples)
    data['rekkos'] = data['userId'].map(personal_rekkos)
    
    return data[['userId', 'rekkos']]

In [102]:
recommended_df = get_recommendatios(data= users,number_of_samples= 20, item_column='watched_movies')
recommended_df.head(5)

,userId,rekkos
0,1,"[Elle boit pas, elle fume pas, elle drague pas..."
1,2,"[Uncle Nick, Happy Burnout, Columbus Day, Whit..."
2,3,"[Uncle Nick, The Southerner, Happy Burnout, Bu..."
3,4,"[Fantozzi Contro Tutti, Vääpeli Körmy ja marsa..."
4,5,"[Vääpeli Körmy ja marsalkan sauva, Inside, The..."


## Adding more features into the model (this model works worse that first one, that is why I used first in recommendations)

In [103]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   movieId                45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [104]:
new_sample = movies_metadata[['movieId', 'original_title', 'overview', 'genres', 'tagline']]

In [105]:
new_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   movieId         45466 non-null  object
 1   original_title  45466 non-null  object
 2   overview        44512 non-null  object
 3   genres          45466 non-null  object
 4   tagline         20412 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [106]:
new_sample.loc[new_sample['overview'].isnull(), 'overview'] = new_sample.loc[new_sample['overview'].isnull(), 'original_title']
new_sample['tagline'] = new_sample['tagline'].fillna(new_sample['original_title'])

In [107]:
new_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   movieId         45466 non-null  object
 1   original_title  45466 non-null  object
 2   overview        45466 non-null  object
 3   genres          45466 non-null  object
 4   tagline         45466 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [108]:
new_sample = new_sample.reset_index().rename(columns = {'index': 'model_index'})
new_sample['model_index'] = new_sample['model_index'].astype(str)
new_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   model_index     45466 non-null  object
 1   movieId         45466 non-null  object
 2   original_title  45466 non-null  object
 3   overview        45466 non-null  object
 4   genres          45466 non-null  object
 5   tagline         45466 non-null  object
dtypes: object(6)
memory usage: 2.1+ MB


In [109]:
movies_inv_mapper = dict(zip(new_sample['original_title'].str.lower(), new_sample['model_index'].astype(int)))

In [110]:
def get_clean_tags_array(agg_tags: pd.DataFrame,
                         text_cols = ['overview', 'original_title', 'tagline']):
    '''text preprocessing
    '''
    # concatenate the text from all columns of interest
    tags_corpus = agg_tags[text_cols].apply(lambda x: ' '.join(x), axis=1).values
    
    # preprocess the text
    tags_corpus = [re.sub('-[!/()0-9]', '', x) for x in tags_corpus]
    stop_words = stopwords.words('english')
    tags_doc = [TaggedDocument(words = word_tokenize_clean(D, stop_words), tags = [str(i)]) for i, D in enumerate(tags_corpus)]

    return tags_doc

In [111]:
tags_array = get_clean_tags_array(new_sample)

In [112]:
def train_embeddings(tags_doc: np.array,
                     epochs = 20,
                     vec_size = 50,
                     alpha = .02,
                     min_alpha =  0.00025,
                     min_count = 5,
                     save_path: str = None):
    """
    fit doc2vec model to prepared corpus
    :tags_doc: result of get_clean_tags_array()
    :max_epocs: int
    :vec_size: int
    :alpha: float
    """
    #initialize
    model = Doc2Vec(vector_size = vec_size,
                    alpha = alpha, 
                    min_alpha = min_alpha,
                    min_count = min_count,
                    dm = 0)
    
    #generate vocab from all tag docs
    model.build_vocab(tags_doc)
    
    #train model
    model.train(tags_doc,
                total_examples = model.corpus_count,
                epochs = epochs)
    
    #save model to dir
    if save_path:
        model.save(f'{save_path}/d2v_model.pkl')
    
    return model

In [113]:
model = train_embeddings(tags_array)

In [114]:
movie_id = movies_inv_mapper['batman']
movie_id

8603

In [115]:
movie_vectors = model.dv.vectors

In [116]:
movie_embeddings = movies_vectors[movie_id]

In [117]:
movie_embeddings

array([-8.0160208e-02, -1.8253161e-01,  2.7860367e-01, -1.9340388e-02,
       -2.6517289e-02,  1.8640238e-01, -3.3289498e-01, -6.1052497e-02,
       -3.5598734e-01, -3.8000790e-04, -1.7075939e-01, -1.1591378e-01,
        7.9556266e-05, -2.0348647e-01,  2.0137224e-01, -9.6725069e-02,
        2.0268382e-01, -1.6914688e-01,  1.5876643e-01, -2.0017324e-01,
        9.4643004e-02,  1.4631657e-01, -3.1319104e-02,  1.7299244e-01,
        1.5535557e-01,  2.1169420e-01, -1.1648132e-01, -1.2777744e-01,
        3.0593607e-01, -2.5232095e-01,  2.5669878e-02,  6.3207977e-02,
       -4.3362007e-02,  7.3174194e-02, -3.5317650e-01,  2.9851297e-01,
       -1.5897123e-02, -7.3791169e-02,  1.8562043e-01,  2.0299160e-01,
        1.6092224e-01, -2.9811222e-02, -3.5844274e-02,  1.4017850e-01,
        1.3455860e-02, -3.6246192e-01,  6.1877277e-02,  2.6866566e-02,
       -1.2715700e-01,  4.0484986e-01], dtype=float32)

In [118]:
similars = model.docvecs.most_similar(positive=[movie_embeddings], topn=20)
output = pd.DataFrame(similars, columns = ['model_index', 'model_score'])
output.head()

,model_index,model_score
0,4276,0.930426
1,10406,0.918615
2,33808,0.917352
3,11186,0.917191
4,36296,0.915977


In [ ]:
output['title_name'] = output['model_index'].astype(int).map(name_mapper)
output

,model_index,model_score,title_name
0,34282,0.915982,kshanbhar vishranti
1,32667,0.912129,some voices
2,13779,0.910338,imagine that
3,24762,0.908032,the returned
4,45022,0.907549,ماجرای نیمروز
5,22855,0.906911,dug's special mission
6,34199,0.906633,der rest ist schweigen
7,34896,0.906432,recep i̇vedik 2
8,19771,0.905578,stolen
9,43461,0.905525,megafault


As we can see, model with more attributes works worse, it doesn't have even batman by itself

In [ ]:
import umap

: 

: 